## Video Intelligence API
Google Cloud Video Intelligence makes videos searchable, and discoverable, by extracting metadata with an easy to use REST API. You can now search every moment of every video file in your catalog. It quickly annotates videos stored in Google Cloud Storage, and helps you identify key entities (nouns) within your video; and when they occur within the video. Separate signal from noise, by retrieving relevant information within the entire video, shot-by-shot, -or per frame.
<br><br><br>
Please [set up a Google Cloud Video Intelligence API project in the Google Cloud Platform Console](https://cloud.google.com/video-intelligence/docs/before-you-begin)

In [1]:
from google.cloud import videointelligence
import json

<br>
## Gather Insight from 20,000 Labels
Cloud Video Intelligence allows developers to extract actionable insights from video files without requiring any machine learning or computer vision knowledge. From our massive library of 20,000 labels, it automatically analyzes video content to identify what entities are in your video content and when they appear. Cloud Video Intelligence improves over time as new concepts are introduced and accuracy is improved.

In [2]:
def analyze_labels(path):
    """ Detects labels given a GCS path. """
    video_client = videointelligence.VideoIntelligenceServiceClient()
    features = [videointelligence.enums.Feature.LABEL_DETECTION]
    operation = video_client.annotate_video(path, features=features)
    print('\nProcessing video for label annotations:')

    result = operation.result(timeout=90)
    print('\nFinished processing.')

    segment_labels = result.annotation_results[0].segment_label_annotations
    for i, segment_label in enumerate(segment_labels):
        print('Video label description: {}'.format(
            segment_label.entity.description))
        for category_entity in segment_label.category_entities:
            print('\tLabel category description: {}'.format(
                category_entity.description))

        for i, segment in enumerate(segment_label.segments):
            start_time = (segment.segment.start_time_offset.seconds +
                          segment.segment.start_time_offset.nanos / 1e9)
            end_time = (segment.segment.end_time_offset.seconds +
                        segment.segment.end_time_offset.nanos / 1e9)
            positions = '{}s to {}s'.format(start_time, end_time)
            confidence = segment.confidence
            print('\tSegment {}: {}'.format(i, positions))
            print('\tConfidence: {}'.format(confidence))
        print('\n')

In [ ]:
analyze_labels('gs://cloud-ml-sandbox/video/chicago.mp4')

## Separate Signals from Noise
You can identify the presence of a signal buried in noise, by using shot detection to distinguish scene changes within a video and discern only relevant entities at the video, shot or frame level (e.g. Identify “dog” within a new scene).

In [4]:
def analyze_shots(path):
    """ Detects camera shot changes. """
    video_client = videointelligence.VideoIntelligenceServiceClient()
    features = [videointelligence.enums.Feature.SHOT_CHANGE_DETECTION]
    operation = video_client.annotate_video(path, features=features)
    print('\nProcessing video for shot change annotations:')

    result = operation.result(timeout=90)
    print('\nFinished processing.')

    for i, shot in enumerate(result.annotation_results[0].shot_annotations):
        start_time = (shot.start_time_offset.seconds +
                      shot.start_time_offset.nanos / 1e9)
        end_time = (shot.end_time_offset.seconds +
                    shot.end_time_offset.nanos / 1e9)
        print('\tShot {}: {} to {}'.format(i, start_time, end_time))

In [8]:
analyze_shots('gs://demomaker/gbikes_dinosaur.mp4')


Processing video for shot change annotations:

Finished processing.
	Shot 0: 0.0 to 5.166666
	Shot 1: 5.233333 to 10.066666
	Shot 2: 10.1 to 28.133333
	Shot 3: 28.166666 to 42.766666


<br><br>
# Bonus Lab: Build Video Search Catalog
Codebase and instructions can be found on [this github repo](https://github.com/sararob/video-intelligence-demo)
If you want to see a [demo on youtube](https://www.youtube.com/watch?v=mDAoLO4G4CQ)
